# Load Data

In [ ]:
!pip install shap

In [1]:
import pandas as pd
from patsy import dmatrix
import numpy as np
from sklearn import tree
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from joblib import Parallel, delayed
import multiprocessing
#import shap

### Read SPXT: market index

Dow Jones, S&P 500 and Nasdaq Composite

I only use spxt sheet, Vix sheet is ignored...


In [2]:
spxt = pd.read_excel('/content/drive/My Drive/SWIB Quant Intern/SPXTVIX.xlsx',sheet_name='SPXT')  

One problem with SPXT, it is not strictly daily, some dates are missing.

In [13]:
spxt[spxt.Dates >= '2001-08-31'].head(10)

,Dates,SPXT
434,2001-08-31,1591.18
435,2001-09-03,1591.18
436,2001-09-04,1590.29
437,2001-09-05,1588.93
438,2001-09-06,1553.41
439,2001-09-07,1524.47
440,2001-09-10,1533.98
441,2001-09-11,1533.98
442,2001-09-12,1533.98
443,2001-09-13,1533.98


Calculate month return and daily return. 

we need daily return to calculate beta, and monthly return for calculate excess return.

In [14]:
spxt['year_month'] = pd.to_datetime(spxt['Dates']).dt.to_period('M')
spxt['d_return'] = np.concatenate(   ([np.nan], [(spxt['SPXT'][t] - spxt['SPXT'][t-1])/spxt['SPXT'][t-1] for t in np.arange(spxt.shape[0]) if t != 0] )   )
spxt['m_return'] = [ ( spxt.loc[spxt['year_month'] == str(T), 'SPXT'].iloc[-1] -  spxt.loc[spxt['year_month'] == str(T), 'SPXT'].iloc[0] )/spxt.loc[spxt['year_month'] == str(T), 'SPXT'].iloc[0] for T in spxt['year_month']] 


In [ ]:
spxt[(spxt.Dates >= '2000-01') & (spxt.Dates < '2000-02') ]

In [21]:
# -0.038310
(1925.41 - 2002.11) / 2002.11
# -0.058829
(6145.30 - 6529.42) / 6529.42

# -0.041092
(1919.84 - 2002.11)/2002.11

-0.041091648311031855

### Read individual stock index

In [22]:
def load_basic_data(fileName):

        tempDF = pd.read_parquet(fileName)
        
        tempDF['date'] = pd.to_datetime(tempDF['date'])
                  
        return tempDF

tempDF = load_basic_data("/content/drive/My Drive/SWIB Quant Intern/SP500.pq")

# Build Signals

For each stock_i where $i \in \{ 1,...1069\}$. we convert daily data to monthly data in order to remove noise. We group daily data by year_month. Suppose month T is the current month.

stock_i_Month_T return = (Price_T.iloc[-1] - Price_T.iloc[0]) / Price_T.iloc[0]

stock_i_month_T_excess_return = stock_i_month_T_return - market_month_T_return


Notes from Roger:

1.   Build those signals based on price, return, volume, volatility, momentum

2.   Use monthly data to remove noise. (first email)

3.   the forecast is for **excess return**, violatility need to be calculated in **return space** not on price. performance to be measured by out of sample R2


4.  Y: next month excess return(T+1).  ?? Should use monthT  excess return??

  Mom1m:  last month excess return (T-1)

  Momentum: previous T-12:T-2 month excess return.

  mom36: last 36 month excess return (T-36 month excess return?).  

  maxret1 : max monthly excess return in the last 12 month

  maxret2 : max daily excess return in the last 1 month

  turnover: dollar volume / market cap

  beta: roll 252 days calculation against index. (return_i = beta * return_m + intercept + epsilon). Beta time-dependent: Beta_i_T I used the lastest available 252 days before last day of monthT.  

  beta^2

  retvol: stdev(daily return)*sqrt(252). ? *sqrt(252)

  idiovol: stdev(daily regression residual)*sqrt(252)




In [27]:
# Map 2001-09 - 2020-06 to T 0 - 225
T = pd.to_datetime(tempDF['date']).dt.to_period('M').unique()
T = dict(zip (T,np.arange(len(T))))

In [ ]:
# for all stock_id
# stock id, total of 1069 stocks 

stock_id = list(tempDF.sedol.unique()) 

stock_month = pd.DataFrame({'year_month':[],'sedol':[],'sector':[],'m_volume':[],'spxt_m_return':[],
 'm_return':[],'excess_return':[],'mom1m':[],'next_month_excess_return':[],'Momentum1':[],'Momentum2':[],
 'Momentum3':[],'Momentum4':[],'Momentum5':[],'Momentum6':[],'Momentum7':[],'Momentum8':[],'Momentum9':[],'Momentum10':[],'Momentum11':[],
 'Momentum12':[],'Momentum36':[],'maxret1':[],'maxret2':[],'dollarVolume':[],'mktcap':[],'turnover':[],'beta':[],'beta_sq':[],'retvol':[],
 'idiovol':[] })


# for stock i, 0:400. 400:800 , 800:1049
for i in np.arange(400):
    print(i)
    # stock_i is daily, which is used for constructing stock_month
    stock_i = tempDF[tempDF['sedol'] == stock_id[i]]; stock_i = stock_i.sort_values(by = ['date']) # make sure the date is correct
    stock_i_sector = stock_i['sector'].unique() # sometimes na + sector
    stock_i['year_month'] = pd.to_datetime(stock_i['date']).dt.to_period('M'); 
    stock_i['date'] = pd.to_datetime(stock_i['date']).dt.to_period('D') # period is better, can + -.
    time_i = list(stock_i['year_month'].unique())

    # SP index in time_i
    #start_m = [str(T) for T in time_i][0] ;end_m = [str(T) for T in time_i][-1] # string. 
    #spxt_i = spxt.loc[start_m:end_m] # 

    # intitial new df, save stock_i monthly data
    stock_i_month = pd.DataFrame(data = {"year_month":time_i})

    # stock id and sector
    stock_i_month['sedol'] = stock_id[i]; stock_i_month['sector'] = stock_i_sector[-1]

    # Since it is VOLUME_AVG_30D, I use the .iloc[-1] of that month to represent that month's volume.
    stock_i_month['m_volume'] = [  stock_i[stock_i['year_month'] == T]['VOLUME_AVG_30D'].iloc[-1]   for T in time_i ]

    # spxt monthly return
    stock_i_month['spxt_m_return'] = [ spxt[spxt['year_month'] == T].m_return.iloc[0]   for T in time_i] # any iloc[.]

    # stock_i monthly return, (price.T.iloc[-1] - price.T.iloc[0] ) / price.T.iloc[0]
    stock_i_month['m_return'] = [ (stock_i[stock_i['year_month'] == T].LAST_PRICE.iloc[-1] - stock_i[stock_i['year_month'] == str(T)].LAST_PRICE.iloc[0])/ 
                                stock_i[stock_i['year_month'] == T].LAST_PRICE.iloc[0]  for T in time_i ]


    # stock_ i excess return
    stock_i_month['excess_return'] = stock_i_month['m_return'] - stock_i_month['spxt_m_return']

    # Mom1m: last month excess return (T-1)
    stock_i_month['mom1m'] = np.concatenate(( [np.nan],stock_i_month['excess_return'].to_list()[0:-1]))

    # next month excess return (T+1), Y 
    stock_i_month['next_month_excess_return'] = np.concatenate( ( stock_i_month['excess_return'].to_list()[1:] , [np.nan]) )

    # Momentum, previous T-12 :  T-2 month excess return. 
    # mom36: last 36 month excess return.
    def empty_series(ele):
        if ele.empty:
          return np.nan
        else:
          return ele.iloc[0]
    for j in np.concatenate((list(np.arange(1,13)),[36])):
        stock_i_month['Momentum'+ str(j)] =  [ empty_series(ele)  for ele in [ stock_i_month[stock_i_month['year_month'] == T -j].excess_return  for T in time_i]  ]

    # maxret1, max monthly excess return in the last 12 month, from T-12 : T
    stock_i_month['maxret1'] = [  stock_i_month[ (stock_i_month['year_month'] >= T -12 ) & (stock_i_month['year_month'] <= T  )].excess_return.max()  for T in time_i]
    # maxret2, max daily excess return in the last 1 month
    stock_i_month['maxret2'] = [ stock_i[stock_i['year_month'] == T].d_return.max()  for T in time_i]

    # turnover: dollarvolume/ marketcap
    # Dollarvolume,  dollarVolume_T.iloc[-1].  which equals to price_T.iloc[-1]* volume_T.iloc[-1]
    # marketcap mktcap_T.mean()
    stock_i_month['dollarVolume'] = [stock_i[stock_i['year_month'] == T].dollarVolume.iloc[-1]  for T in time_i]
    stock_i_month['mktcap'] = [ stock_i[stock_i['year_month'] == T].mktcap.mean()    for T in time_i]
    stock_i_month['turnover'] = stock_i_month['dollarVolume']/stock_i_month['mktcap']
    
    # beta, roll 252 days , r_i ~ beta r_m.
    beta = list()
    retvol = list()
    idiovol = list()
    for T in time_i:
      last_date = stock_i[stock_i['year_month'] == T].date.iloc[-1]
      start_date = last_date - 252
      # daily data for stock_i, for the most recent 252 days, however, maybe not available up to 252 days.
      stock_i_T = stock_i.loc[ (stock_i['date'] >= start_date) &  (stock_i['date'] <= last_date), ['date','sedol','d_return']]
      # def empty_series, if at date_t, spxt[ spxt['Dates'] == str(t) ].d_return maybe empty series, then iloc will get error
      stock_i_T['spxt_d_return'] = [empty_series(ele)  for ele in [ spxt[ spxt['Dates'] == str(t) ].d_return  for t in stock_i_T['date']  ] ]
      # retvol: std(d_return), for the most recent 252 days, however, maybe not available up to 252 days.
      retvol.append(stock_i_T['d_return'].std())
      # run linear regression r_i ~ r_m
      reg = LinearRegression().fit(stock_i_T[['d_return']],stock_i_T[['spxt_d_return']] )
      beta.append(reg.coef_[0][0])
      # residual
      predicted = reg.predict(stock_i_T[['spxt_d_return']]).reshape((-1,))
      residual = stock_i_T.d_return.to_numpy() - predicted
      idiovol.append(np.std(residual))

    stock_i_month['beta'] = beta
    stock_i_month['beta_sq'] = [ele**2 for ele in beta]
    stock_i_month['retvol'] = retvol
    stock_i_month['idiovol'] = idiovol

    # merge stock_i to previous DF
    stock_month = pd.concat([stock_month,stock_i_month])


# Save stock_month as .pkl
# It runs slowly So I partitioned the job and merge them
# stock_month.to_pickle('/content/drive/My Drive/SWIB Quant Intern/stock_month.pkl')



In [28]:
# Stock_month final data already saved, so directly load

stock_month = pd.read_pickle('/content/drive/My Drive/SWIB Quant Intern/stock_month.pkl')

# map T: 2001-06: 0,    2020-06 :226
stock_month['Month_T'] = stock_month['year_month'].map(T)

In [29]:
stock_month

,year_month,sedol,sector,m_volume,spxt_m_return,m_return,excess_return,mom1m,next_month_excess_return,Momentum1,Momentum2,Momentum3,Momentum4,Momentum5,Momentum6,Momentum7,Momentum8,Momentum9,Momentum10,Momentum11,Momentum12,Momentum36,maxret1,maxret2,dollarVolume,mktcap,turnover,beta,beta_sq,retvol,idiovol,Month_T
0,2001-09,2001692,Industrials,2605739.0,-0.080751,0.063333,0.144085,NaN,-0.106841,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.144085,0.111111,4.987384e+07,2880.790110,17312.557512,0.132487,0.017553,0.137694,0.128356,0
1,2001-10,2001692,Industrials,1800316.5,0.021414,-0.085427,-0.106841,0.144085,0.095953,0.144085,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.144085,0.070150,3.276576e+07,3103.578478,10557.413170,0.144685,0.020934,0.081290,0.078393,1
2,2001-11,2001692,Industrials,2351187.0,0.052434,0.148387,0.095953,-0.106841,0.047668,-0.106841,0.144085,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.144085,0.069867,5.022135e+07,2997.686324,16753.372066,0.139212,0.019380,0.067879,0.065963,2
3,2001-12,2001692,Industrials,1609172.7,0.017280,0.064947,0.047668,0.095953,0.152369,0.095953,-0.106841,0.144085,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.144085,0.058984,3.588455e+07,3427.028595,10471.039332,0.139066,0.019339,0.059273,0.057752,3
4,2002-01,2001692,Industrials,1856330.4,-0.014596,0.137774,0.152369,0.047668,0.094381,0.047668,0.095953,-0.106841,0.144085,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.152369,0.078467,4.629688e+07,3851.456852,12020.615043,0.136304,0.018579,0.054357,0.053085,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1,2020-06,B01SD70,Consumer Discretionary,1211380.5,-0.016985,-0.018692,-0.001708,-0.056847,NaN,-0.056847,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.001708,0.013719,4.559757e+08,15003.818400,30390.646024,-0.115460,0.013331,0.023628,0.022708,225
0,2020-05,B0796X4,Health Care,2609637.2,0.077750,-0.079090,-0.156839,NaN,-0.020538,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.156839,0.045546,9.872518e+08,36521.904077,27031.773783,-0.044269,0.001960,0.030482,0.029235,224
1,2020-06,B0796X4,Health Care,2606988.0,-0.016985,-0.037523,-0.020538,-0.156839,NaN,-0.156839,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.020538,0.056952,9.502211e+08,33775.340000,28133.574854,0.061081,0.003731,0.032742,0.031711,225
0,2020-05,2950482,Health Care,1319074.0,0.077750,0.021756,-0.055994,NaN,-0.013231,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.055994,0.048585,2.849727e+08,15320.304800,18600.984163,-0.156655,0.024541,0.031523,0.027512,224


In [30]:
stock_month.groupby('sedol').beta.agg(['mean','std','size']).sort_values(by = 'mean')

,mean,std,size
sedol,,,
2435008,-0.263375,0.076134,4
2691725,-0.199199,0.193810,11
2024677,-0.191939,0.133102,11
2950482,-0.118878,0.053425,2
B01SD70,-0.115245,0.000304,2
...,...,...,...
BK6YKG1,0.656716,0.061558,11
2339638,0.694913,0.153723,34
B3QN1M2,0.699902,NaN,1


### Paralell

I Tried to parallel the for loop, However, I didn't see improved performance.

In [ ]:
# for all stock_id
# stock id, total of 1069 stocks 

stock_id = list(tempDF.sedol.unique()) 

def paralell_func(i,tempDF = tempDF,spxt = spxt,stock_id = stock_id):#,tempDF,spxt,stock_id):
    print(i)
    # stock_i is daily, which is used for constructing stock_month
    stock_i = tempDF[tempDF['sedol'] == stock_id[i]]; stock_i = stock_i.sort_values(by = ['date']) # make sure the date is correct
    stock_i_sector = stock_i['sector'].unique() # sometimes na + sector
    stock_i['year_month'] = pd.to_datetime(stock_i['date']).dt.to_period('M'); 
    stock_i['date'] = pd.to_datetime(stock_i['date']).dt.to_period('D') # period is better, can + -.
    time_i = list(stock_i['year_month'].unique())

    # SP index in time_i
    #start_m = [str(T) for T in time_i][0] ;end_m = [str(T) for T in time_i][-1] # string. 
    #spxt_i = spxt.loc[start_m:end_m] # 

    # intitial new df, save stock_i monthly data
    stock_i_month = pd.DataFrame(data = {"year_month":time_i})

    # stock id and sector
    stock_i_month['sedol'] = stock_id[i]; stock_i_month['sector'] = stock_i_sector[-1]

    # Since it is VOLUME_AVG_30D, I use the .iloc[-1] of that month to represent that month's volume.
    stock_i_month['m_volume'] = [  stock_i[stock_i['year_month'] == T]['VOLUME_AVG_30D'].iloc[-1]   for T in time_i ]

    # spxt monthly return
    stock_i_month['spxt_m_return'] = [ spxt[spxt['year_month'] == T].m_return.iloc[0]   for T in time_i] # any iloc[.]

    # stock_i monthly return, (price.T.iloc[-1] - price.T.iloc[0] ) / price.T.iloc[0]
    stock_i_month['m_return'] = [ (stock_i[stock_i['year_month'] == T].LAST_PRICE.iloc[-1] - stock_i[stock_i['year_month'] == str(T)].LAST_PRICE.iloc[0])/ 
                                stock_i[stock_i['year_month'] == T].LAST_PRICE.iloc[0]  for T in time_i ]


    # stock_ i excess return
    stock_i_month['excess_return'] = stock_i_month['m_return'] - stock_i_month['spxt_m_return']

    # Mom1m: last month excess return (T-1)
    stock_i_month['mom1m'] = np.concatenate(( [np.nan],stock_i_month['excess_return'].to_list()[0:-1]))

    # next month excess return (T+1), Y 
    stock_i_month['next_month_excess_return'] = np.concatenate( ( stock_i_month['excess_return'].to_list()[1:] , [np.nan]) )

    # Momentum, previous T-12 :  T-2 month excess return. 
    # mom36: last 36 month excess return.
    def empty_series(ele):
        if ele.empty:
          return np.nan
        else:
          return ele.iloc[0]
    for j in np.concatenate((list(np.arange(1,13)),[36])):
        stock_i_month['Momentum'+ str(j)] =  [ empty_series(ele)  for ele in [ stock_i_month[stock_i_month['year_month'] == T -j].excess_return  for T in time_i]  ]

    # maxret1, max monthly excess return in the last 12 month, from T-12 : T
    stock_i_month['maxret1'] = [  stock_i_month[ (stock_i_month['year_month'] >= T -12 ) & (stock_i_month['year_month'] <= T  )].excess_return.max()  for T in time_i]
    # maxret2, max daily excess return in the last 1 month
    stock_i_month['maxret2'] = [ stock_i[stock_i['year_month'] == T].d_return.max()  for T in time_i]

    # turnover: dollarvolume/ marketcap
    # Dollarvolume,  dollarVolume_T.iloc[-1].  which equals to price_T.iloc[-1]* volume_T.iloc[-1]
    # marketcap mktcap_T.mean()
    stock_i_month['dollarVolume'] = [stock_i[stock_i['year_month'] == T].dollarVolume.iloc[-1]  for T in time_i]
    stock_i_month['mktcap'] = [ stock_i[stock_i['year_month'] == T].mktcap.mean()    for T in time_i]
    stock_i_month['turnover'] = stock_i_month['dollarVolume']/stock_i_month['mktcap']
    
    # beta, roll 252 days , r_i ~ beta r_m.
    beta = list()
    retvol = list()
    idiovol = list()
    for T in time_i:
      last_date = stock_i[stock_i['year_month'] == T].date.iloc[-1]
      start_date = last_date - 252
      # daily data for stock_i, for the most recent 252 days, however, maybe not available up to 252 days.
      stock_i_T = stock_i.loc[ (stock_i['date'] >= start_date) &  (stock_i['date'] <= last_date), ['date','sedol','d_return']]
      # def empty_series, if at date_t, spxt[ spxt['Dates'] == str(t) ].d_return maybe empty series, then iloc will get error
      stock_i_T['spxt_d_return'] = [empty_series(ele)  for ele in [ spxt[ spxt['Dates'] == str(t) ].d_return  for t in stock_i_T['date']  ] ]
      # retvol: std(d_return), for the most recent 252 days, however, maybe not available up to 252 days.
      retvol.append(stock_i_T['d_return'].std())
      # run linear regression r_i ~ r_m
      reg = LinearRegression().fit(stock_i_T[['d_return']],stock_i_T[['spxt_d_return']] )
      beta.append(reg.coef_[0][0])
      # residual
      predicted = reg.predict(stock_i_T[['spxt_d_return']]).reshape((-1,))
      residual = stock_i_T.d_return.to_numpy() - predicted
      idiovol.append(np.std(residual))

    stock_i_month['beta'] = beta
    stock_i_month['beta_sq'] = [ele**2 for ele in beta]
    stock_i_month['retvol'] = retvol
    stock_i_month['idiovol'] = idiovol

    # merge stock_i to previous DF
    return stock_i_month

num_cores = multiprocessing.cpu_count()
     
results = Parallel(n_jobs=num_cores)(delayed(paralell_func)(i) for i in np.arange(len(stock_id[800:1068])) )

# Build Prediction Model

let $T \in \{1,2,....,226 \}$ denote the month index, 
let $ i \in \{1,2,....,1069 \}$ denote the stock index. 
The response variable is Next_month_excess_return, denoted as $r_{i,T+1}$, of stock i and month T+1. 
The predictive characteristics are denoted as P-dimensional vector $z_{i,T}$, 
T is 1,2....226 following "citi-bank" paper, which measures the overal time trend for the macromarket.

1.  we consider regression problem, which means that the objective function is minimization of MSE. 

   $r_{i,T+1} = g(z_{i,T}) + \epsilon_{i,T+1}$
   
       

2.  we consider classification problem, for example a binary classification of whether d_return is positive or not. 
   
   $P(r_{i,T+1} > 0 | z_{i,T}) = g(z_{i,T})$

Following the paper(ML), The g function depends neither on i or T. By maintaining the same form over time and across different stocks, the model leverages information from the entire panel and every individual asset. Also g(.) depends on z only through $z_{i,T}$.


We split sample into 2 disjoint time periods, 2001-2013 as training, 2014-2020 as testing.

Regarding performance evaluation for regression, we use R square $R^2_{oos} = 1 - \frac{\sum_{(i,T)\in test}(r_{i,T+1}-\hat{r}_{i,T+1})^2}{\sum_{(i,T)\in test} r^2_{i,T+1}}$, 
and F1,accuracy for classification.






How to handle missing data? 

I remove all rows contains NA, data size drop from 113594 to 78857, only 817 stocks left(1069). 

In [33]:
stock_month_dropna = stock_month.dropna()
stock_month_dropna

,year_month,sedol,sector,m_volume,spxt_m_return,m_return,excess_return,mom1m,next_month_excess_return,Momentum1,Momentum2,Momentum3,Momentum4,Momentum5,Momentum6,Momentum7,Momentum8,Momentum9,Momentum10,Momentum11,Momentum12,Momentum36,maxret1,maxret2,dollarVolume,mktcap,turnover,beta,beta_sq,retvol,idiovol,Month_T
36,2004-09,2002305,Health Care,3007200.00,0.008978,0.007133,-0.001846,0.062294,0.010333,0.062294,-0.010611,-0.025069,-0.030706,0.072736,-0.028479,-0.025834,-0.093892,-0.004894,0.031828,-0.037129,0.066452,0.106874,0.072736,0.015100,1.273850e+08,66315.109545,1920.904495,0.278098,0.077338,0.012023,0.011196,36
37,2004-10,2002305,Health Care,3714204.30,0.000096,0.010429,0.010333,-0.001846,-0.058427,-0.001846,0.062294,-0.010611,-0.025069,-0.030706,0.072736,-0.028479,-0.025834,-0.093892,-0.004894,0.031828,-0.037129,-0.017815,0.072736,0.035400,1.583365e+08,64877.967143,2440.528523,0.297073,0.088252,0.011777,0.010865,37
38,2004-11,2002305,Health Care,3872917.50,0.040177,-0.018250,-0.058427,0.010333,0.060134,0.010333,-0.001846,0.062294,-0.010611,-0.025069,-0.030706,0.072736,-0.028479,-0.025834,-0.093892,-0.004894,0.031828,-0.032404,0.072736,0.030700,1.625076e+08,67803.075909,2396.758792,0.291366,0.084894,0.011743,0.010853,38
39,2004-12,2002305,Health Care,3524317.40,0.018479,0.078613,0.060134,-0.058427,-0.018951,-0.058427,0.010333,-0.001846,0.062294,-0.010611,-0.025069,-0.030706,0.072736,-0.028479,-0.025834,-0.093892,-0.004894,-0.012051,0.072736,0.041500,1.644094e+08,70394.793043,2335.533633,0.295938,0.087580,0.010747,0.009925,39
40,2005-01,2002305,Health Care,3987872.80,-0.016404,-0.035355,-0.018951,0.060134,-0.004392,0.060134,-0.058427,0.010333,-0.001846,0.062294,-0.010611,-0.025069,-0.030706,0.072736,-0.028479,-0.025834,-0.093892,0.047905,0.072736,0.022600,1.795340e+08,71976.725238,2494.334562,0.285524,0.081524,0.010750,0.009980,40
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38,2020-05,2718992,Financials,1005670.80,0.077750,0.096550,0.018800,-0.062620,0.159561,-0.062620,-0.112622,-0.001640,0.007552,-0.025518,0.028181,0.017219,0.053312,0.009686,-0.068509,-0.043093,-0.020840,-0.048072,0.053312,0.078669,6.967287e+07,8959.712850,7776.239506,0.518964,0.269324,0.038649,0.028981,224
36,2020-04,BYXD7B3,Information Technology,4785628.00,0.180287,0.515886,0.335599,-0.287955,-0.231413,-0.287955,-0.161601,-0.132361,-0.002721,0.289535,-0.048893,-0.083390,-0.390421,-0.009680,0.109025,-0.221468,-0.027057,0.073708,0.335599,0.158526,8.676344e+07,3829.012148,22659.482993,0.252332,0.063672,0.064797,0.060651,223
37,2020-05,BYXD7B3,Information Technology,4824443.00,0.077750,-0.153663,-0.231413,0.335599,0.236497,0.335599,-0.287955,-0.161601,-0.132361,-0.002721,0.289535,-0.048893,-0.083390,-0.390421,-0.009680,0.109025,-0.221468,0.002345,0.335599,0.094895,6.855534e+07,4119.540900,16641.498821,0.248950,0.061976,0.066986,0.062801,224
36,2020-04,2372763,Information Technology,684272.64,0.180287,0.273556,0.093269,-0.070789,-0.018752,-0.070789,-0.121372,0.030769,-0.061447,0.001705,0.059560,0.060785,-0.020214,-0.155251,0.000571,0.023375,-0.002836,0.032611,0.093269,0.087563,8.129843e+07,9373.477152,8673.241641,0.554350,0.307303,0.030590,0.023031,223


In [39]:
len(stock_month_dropna.sedol.unique())

817

In [ ]:
T

In [64]:
# split data, remember after remove NAs, T starts from 36 to 225. 
# training 36: 180,   testing 180:225

features = [ 'Month_T', 'mom1m', 'Momentum2', 'Momentum3', 'Momentum4', 'Momentum5',
       'Momentum6', 'Momentum7', 'Momentum8', 'Momentum9', 'Momentum10',
       'Momentum11', 'Momentum12', 'Momentum36', 'maxret1', 'maxret2',
        'turnover', 'beta', 'beta_sq', 'retvol',
       'idiovol']


X_train = stock_month_dropna.loc[ stock_month_dropna.Month_T.isin(np.arange(0,180)) , features] ## 60477 all 78857
Y_train = stock_month_dropna.loc[ stock_month_dropna.Month_T.isin(np.arange(0,180)) , 'next_month_excess_return']

X_test = stock_month_dropna.loc[ stock_month_dropna.Month_T.isin(np.arange(180,226)) , features] # 18380, match!
Y_test = stock_month_dropna.loc[ stock_month_dropna.Month_T.isin(np.arange(180,226)) , 'next_month_excess_return']


In [70]:
# build model
regr = RandomForestRegressor(max_depth=3,random_state=0)
regr.fit(X_train, Y_train)
regr.score(X_train,Y_train)

0.05694972469073167

In [76]:
# Classification model
# Y to binary 
def pos_ind(a):
  return 1 if a > 0 else 0 
Y_train_bi = [pos_ind(ele) for ele in Y_train]
Y_test_bi = [pos_ind(ele) for ele in Y_test]

clf = RandomForestClassifier(max_depth= 3,random_state=0)
clf.fit(X_train, Y_train_bi)
#clf.score(X_test,Y_test)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=3, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=0, verbose=0,
                       warm_start=False)

In [79]:
print(clf.score(X_test,Y_test_bi))
clf.score(X_train,Y_train_bi)


0.5145266594124048


0.5287133951750252